In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import joblib
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error



In [2]:

%run ploting_02.ipynb

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 16 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   student_id                     1000 non-null   object 
 1   age                            1000 non-null   int64  
 2   gender                         1000 non-null   object 
 3   study_hours_per_day            1000 non-null   float64
 4   social_media_hours             1000 non-null   float64
 5   netflix_hours                  1000 non-null   float64
 6   part_time_job                  1000 non-null   object 
 7   attendance_percentage          1000 non-null   float64
 8   sleep_hours                    1000 non-null   float64
 9   diet_quality                   1000 non-null   object 
 10  exercise_frequency             1000 non-null   int64  
 11  parental_education_level       909 non-null    object 
 12  internet_quality               1000 non-null   ob

,student_id,age,gender,study_hours_per_day,social_media_hours,netflix_hours,part_time_job,attendance_percentage,sleep_hours,diet_quality,exercise_frequency,parental_education_level,internet_quality,mental_health_rating,extracurricular_participation,exam_score,study_sleep_ratio(newlly added
0,S1000,23,Female,0.0,1.2,1.1,No,85.0,8.0,Fair,6,Master,Average,8,Yes,56.2,0.000000
1,S1001,20,Female,6.9,2.8,2.3,No,97.3,4.6,Good,6,High School,Average,8,No,100.0,1.500000
2,S1002,21,Male,1.4,3.1,1.3,No,94.8,8.0,Poor,1,High School,Poor,1,No,34.3,0.175000
3,S1003,23,Female,1.0,3.9,1.0,No,71.0,9.2,Poor,4,Master,Good,1,Yes,26.8,0.108696
4,S1004,19,Female,5.0,4.4,0.5,No,90.9,4.9,Fair,3,Master,Good,1,No,66.4,1.020408


In [4]:
from sklearn.preprocessing import LabelEncoder
df = df.copy()

 #Create derived feature
df['study_sleep_ratio'] = df['study_hours_per_day'] / (df['sleep_hours'] + 1e-6)

# Select ONLY important features
features = [
    'study_hours_per_day',
    'sleep_hours',
    'social_media_hours',
    'attendance_percentage',
    'mental_health_rating',
    'exercise_frequency',
    'study_sleep_ratio'
]

X = df[features]
Y = df['exam_score']

X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.2, random_state=42
)
scaler=StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)
print(X_train.head())
print(X_train_scaled)
joblib.dump(scaler,'E:\Python\Mini Project\models\scaler.joblib') 
load=joblib.load('E:\Python\Mini Project\models\scaler.joblib')



     study_hours_per_day  sleep_hours  social_media_hours  \
29                   2.0          6.7                 3.2   
535                  6.4          6.1                 4.2   
695                  2.5          7.0                 1.9   
557                  3.8          7.3                 3.6   
836                  3.5          3.8                 2.8   

     attendance_percentage  mental_health_rating  exercise_frequency  \
29                    82.6                    10                   6   
535                   88.8                     6                   0   
695                   93.6                     8                   4   
557                  100.0                     7                   1   
836                   92.0                     5                   5   

     study_sleep_ratio  
29            0.298507  
535           1.049180  
695           0.357143  
557           0.520548  
836           0.921052  
[[-1.04978529e+00  1.91900177e-01  6.12600904e-01 

In [5]:
lr=LinearRegression()
lr.fit(X_train_scaled,Y_train)

LinearRegression()

In [26]:
y_predit=lr.predict(X_test_scaled)

In [6]:
import numpy as np
print("R2 Score:", r2_score(Y_test, y_predit))
rmse=np.sqrt(mean_squared_error(Y_test, y_predit))
print("RMSE:",rmse )


NameError: name 'y_predit' is not defined

In [7]:
joblib.dump(lr,'E:\Python\Mini Project\models\student_model_lr.joblib')

['E:\\Python\\Mini Project\\models\\student_model_lr.joblib']

In [29]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train_scaled, Y_train)
pred = rf.predict(X_test_scaled)
rmse=np.sqrt(mean_squared_error(Y_test, pred))
print("R2 Score:", r2_score(Y_test, pred))
print("RMSE:", rmse)

joblib.dump(rf, "E:\Python\Mini Project\models\student_model_rf.joblib")

R2 Score: 0.8466248525479338
RMSE: 6.271351561665158


['E:\\Python\\Mini Project\\models\\student_model_rf.joblib']

In [20]:
from sklearn.model_selection import cross_val_score

lr_cv_scores = cross_val_score(lr, X_train_scaled, Y_train, cv=5, scoring='r2')
print("LR Cross-validation R2:", lr_cv_scores.mean())
rf_cv_scores = cross_val_score(rf, X_train, Y_train, cv=5, scoring='r2')
print("RF Cross-validation R2:", rf_cv_scores.mean())


LR Cross-validation R2: 0.8964567820259071
RF Cross-validation R2: 0.8443090174324952


In [30]:
features = X_train.columns

for f, imp in zip(features, rf.feature_importances_):
    print(f"{f}: {imp:.3f}")


age: 0.008
study_hours_per_day: 0.697
social_media_hours: 0.037
netflix_hours: 0.035
part_time_job: 0.002
attendance_percentage: 0.020
sleep_hours: 0.033
diet_quality: 0.004
exercise_frequency: 0.025
parental_education_level: 0.004
internet_quality: 0.004
mental_health_rating: 0.105
extracurricular_participation: 0.002
study_sleep_ratio(newlly added: 0.022
gender_Male: 0.002
gender_Other: 0.000
